In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import random
from collections import deque

from network import *
from evaluation import *

random.seed(261)

## Simulation 1
3-cycle with specific transition matrix

In [3]:
transition_matrix = np.array([[0, 1/3, 0], [0, 0, 1/3], [1/3, 0, 0]])

In [4]:
NUM_NODES = 3
NUM_SIMS = 100
NUM_ITERS = 10000

In [5]:
all_sims, _ = run_simulation(NUM_NODES, NUM_SIMS, NUM_ITERS, transition_matrix)

Sim 050
Sim 100


In [6]:
vec_list_3 = [
    '[1 1 1]',
    '[0 2 1]',
    '[2 1 0]',
    '[1 0 2]',
    '[2 0 1]',
    '[0 1 2]',
    '[1 2 0]']

In [7]:
all_freqs = [get_cumprops(states, vec_list_3) for states in all_sims]
all_longs = pd.concat([make_long(cumprop, i, vec_list_3) for (i, cumprop) in enumerate(all_freqs)])
all_longs.to_csv("all_longs.csv")

## Simulation 2

3-cycle with randomly generated transition matrices

In [8]:
NUM_NODES = 3
NUM_SIMS = 200
NUM_ITERS = 10000

In [9]:
all_sims, all_trans = run_simulation(NUM_NODES, NUM_SIMS, NUM_ITERS)

Sim 050
Sim 100
Sim 150
Sim 200


In [10]:
all_freqs = [get_cumprops(states, vec_list_3) for states in all_sims]
all_longs = pd.concat([make_long(cumprop, i, vec_list_3) for (i, cumprop) in enumerate(all_freqs)])

In [11]:
final_distribs = all_longs.loc[all_longs["iter"] == NUM_ITERS].copy()
final_distribs["estimated"] = np.concatenate([get_distrib(make_p_matrix(tm)) for tm in all_trans])
final_distribs.to_csv("final_distribs.csv")

In [12]:
sim_metrics = final_distribs.groupby("sim").agg({'value': calc_entropy, 'estimated': calc_entropy})
sim_metrics["rmse"] = final_distribs.groupby("sim").apply(lambda x: calc_rmse(x["value"], x["estimated"]))
sim_metrics["symmetry"] = [calc_symmetry(tm) for tm in all_trans]
sim_metrics.to_csv("sim_metrics_3.csv")

## Simulation 3

K5 with randomly generated transition matrices

In [13]:
NUM_NODES = 5
NUM_SIMS = 500
NUM_ITERS = 5000

In [14]:
all_sims, all_trans = run_simulation(NUM_NODES, NUM_SIMS, NUM_ITERS)

Sim 050
Sim 100
Sim 150
Sim 200
Sim 250
Sim 300
Sim 350
Sim 400
Sim 450
Sim 500


In [15]:
vec_list_5 = np.load("k5_vecs.npy")

In [16]:
all_freqs = [get_cumprops(states, vec_list_5) for states in all_sims]
all_longs = pd.concat([make_long(cumprop, i, vec_list_5) for (i, cumprop) in enumerate(all_freqs)])

In [19]:
final_distribs = all_longs.loc[all_longs["iter"] == NUM_ITERS].copy()

In [20]:
sim_metrics = final_distribs.groupby("sim").agg({'value': calc_entropy})
sim_metrics["symmetry"] = [calc_symmetry(tm) for tm in all_trans]
sim_metrics.to_csv("sim_metrics_5.csv")

/Users/alvintan/miniconda3/envs/cs330hw0/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
